```
bpset B199 - VGJSRL
B15A - SCARNG
DF6C - VGSCAL
bpset DF57 - VGADD2

$74 - VGLIST
```

In [109]:
a = """
          X - X coordinate bits
          Y - Y coordinate bits
          I - Vector intensity(brightness) bits
          A - Vector address bits
          R - Red color bit
          G - Green color bit
          B - Blue color bit
          S - Scale (size)
          U - Unused bit
 Opcode                     Hex      Binary
    Draw relative vector.      0x00     000YYYYY YYYYYYYY IIIXXXXX XXXXXXXX
    Halt                       0x20     00100000 00000000
    Draw short relative vector 0x40     010YYYYY IIIXXXXX
    New color/intensity        0x60     0110URGB IIIIIIII
    New scale                  0x70     0111USSS SSSSSSSS
    Center                     0x80     10000000 00000000
    Jump to subroutine         0xA0     101AAAAA AAAAAAAA
    Return from subroutine     0xC0     11000000 00000000
    Jump to new address        0xE0     111AAAAA AAAAAAAA
opcode_map = {
 "0": (draw_relative_vector,3),
 "2": (halt,1),
 "4": (draw_short_relative_vector,1),
 "6": (new_color,1),
 "7": (new_scale,1),
 "8": (center,1),
 "A": (jump_to_subroutine,1),
 "C": (return_from_subroutine,1),
 "E": (jump_to_new_address,1),
}
"""


### Recreate Appearance of Tempest Rainbow

In [102]:
from PIL import Image
import PIL
import numpy as np
import sys
#ImageColor.getrgb(color)

colors = [
    "white",
    "yellow","yellow","yellow",
    "magenta","magenta","magenta","magenta",
    "red","red","red","red",
    "cyan","cyan","cyan","cyan",
    "chartreuse","chartreuse","chartreuse","chartreuse",
         ]
def tempest_with_color(color):
    img = Image.open("titles/tempest_clean.png")
    img = img.convert('RGBA')
    pixels = img.load()
    width,height = img.size
    for y in range(height):
        for x in range(width):
            if pixels[x,y] == (255,255,255,255):
                pixels[x,y] = PIL.ImageColor.getrgb(color)
    return img

width,height = Image.open("titles/tempest_clean.png").size
scale = 1
offset_x = 60
offset_y = 40
rainbow_imgs = []
for color in colors:
    img = tempest_with_color(color)
    img.thumbnail([width*scale, sys.maxsize], Image.LANCZOS)
    scale -=0.03
    offset_x += 10
    offset_y += 10
    rainbow_imgs += [(img, offset_x, offset_y)]


In [107]:
def display_rainbow_iterations(num):
    new_img = Image.new( 'RGB', (width+100,height+150), "black")
    for img,offset_x,offset_y in reversed(rainbow_imgs[:num]):
        new_img.paste(img,(offset_x,offset_y),img)
    return new_img

for i,_ in enumerate(colors):
    img = display_rainbow_iterations(i+1)
    img.save(f"titles/rainbow/{i}.png")
    

In [115]:
a = """
 Opcode                     Hex      Binary
    Draw relative vector.      0x00     000YYYYY YYYYYYYY IIIXXXXX XXXXXXXX
    Halt                       0x20     00100000 00000000
    Draw short relative vector 0x40     010YYYYY IIIXXXXX
    New color/intensity        0x60     0110URGB IIIIIIII
    New scale                  0x70     0111USSS SSSSSSSS
    Center                     0x80     10000000 00000000
    Jump to subroutine         0xA0     101AAAAA AAAAAAAA
    Return from subroutine     0xC0     11000000 00000000
    Jump to new address        0xE0     111AAAAA AAAAAAAA
"""

colors = [
    "white",
    "yellow","yellow","yellow",
    "magenta","magenta","magenta","magenta",
    "red","red","red","red",
    "cyan","cyan","cyan","cyan",
    "chartreuse","chartreuse","chartreuse","chartreuse",
         ]

colors = {
    "0": "White",
    "1": "Yellow",
    "2": "Magenta",
    "3": "Red",
    "4": "Cyan",
    "5": "Green",
}
vector_opcodes = [x.strip() for x in ' '.join("""
7220  6800  AFA7  7228  6801  AFA7  7230
6801  AFA7  7238  6801  AFA7  7240  6802  AFA7
7248  6802  AFA7  7250  6802  AFA7  7258  6802
AFA7  7260  6803  AFA7  7268  6803  AFA7  7270
6803  AFA7  7278  6803  AFA7  7300  6804  AFA7
7308  6804  AFA7  7310  6804  AFA7  7318  6804
AFA7  7320  6805  AFA7  7328  6805  AFA7  7330
6805  AFA7
""".split('\n')).split(' ') if x]
vector_list = [vector_opcodes[i:i+3] for i in range(0,len(vector_opcodes),3)]



In [131]:
header = """\\begin{figure}[H]
  {
    \\setlength{\\tabcolsep}{3.0pt}
    \\setlength\\cmidrulewidth{\\heavyrulewidth} % Make cmidrule = 
    \\begin{adjustbox}{center}
      \\begin{tabular}[t]{lll}
        \\toprule
        Op Code & Parsed Data & Written to Screen\\\\
"""

footer = """        \\bottomrule
      \\end{tabular}
    \\end{adjustbox}
  }
\\end{figure}
"""

In [132]:
for i,(op1,op2,op3) in enumerate(vector_list):
    if i % 6 == 0:
        if i > 0:
            print(footer)
        print(header)
    row_template = f"""        \\midrule
          \\icode{{{op1}}}
        &
        \\makecell[tl]{{
          \\begingroup
          \\renewcommand{{\\arraystretch}}{{.8}} % Default value: 1
          \\begin{{tabular}}[t]{{ll}}
            Set Scale: & \\icode{{{op1[1:]}}}\\\\
            \\icode{{}} &  \\\\
          \\end{{tabular}}
          \\endgroup
        }} &
        \\multirow{{3}}{{*}}[-0.1cm]{{
          \\makecell[tl]{{
            \\includegraphics[height=2.3cm]{{titles/rainbow/{i}.png}}%
          }}
        }} \\\\
        \\addlinespace
          \\icode{{{op2}}}
        &
        \\makecell[tl]{{
          \\begingroup
          \\renewcommand{{\\arraystretch}}{{.8}} % Default value: 1
          \\begin{{tabular}}[t]{{ll}}
            New Color: & {colors[op2[-1:]]}\\\\
            Intensity: &  \\icode{{00}}\\\\
          \\end{{tabular}}
          \\endgroup
        }} &
        \\\\
        \\addlinespace
          \\icode{{AFA7}}
        &
        \\makecell[tl]{{
          \\begingroup
          \\renewcommand{{\\arraystretch}}{{.8}} % Default value: 1
          \\begin{{tabular}}[t]{{ll}}
            Jump to Routine: & \\icode{{VORLIT}}  \\\\
            \\icode{{}} &  \\\\
          \\end{{tabular}}
          \\endgroup
        }} &
        \\\\
"""
    print(row_template)
print(footer)


\begin{figure}[H]
  {
    \setlength{\tabcolsep}{3.0pt}
    \setlength\cmidrulewidth{\heavyrulewidth} % Make cmidrule = 
    \begin{adjustbox}{center}
      \begin{tabular}[t]{lll}
        \toprule
        Op Code & Parsed Data & Written to Screen\\

        \midrule
          \icode{7220}
        &
        \makecell[tl]{
          \begingroup
          \renewcommand{\arraystretch}{.8} % Default value: 1
          \begin{tabular}[t]{ll}
            Set Scale: & \icode{220}\\
            \icode{} &  \\
          \end{tabular}
          \endgroup
        } &
        \multirow{3}{*}[-0.1cm]{
          \makecell[tl]{
            \includegraphics[height=2.3cm]{titles/rainbow/0.png}%
          }
        } \\
        \addlinespace
          \icode{6800}
        &
        \makecell[tl]{
          \begingroup
          \renewcommand{\arraystretch}{.8} % Default value: 1
          \begin{tabular}[t]{ll}
            New Color: & White\\
            Intensity: &  \icode{00}\\
          \end{tabula

### Scratchpad

#### Clean up the image first

In [37]:
# Imports PIL module
from PIL import Image
import PIL
import numpy as np
import sys

img = Image.open("titles/tempest.png")
img = im.convert('RGBA')
pixels = img.load()
width,height = img.size
for y in range(height):
    for x in range(width):
        if pixels[x,y] == (255,255,255,255):
            pixels[x,y] = (255,255,255,255)
        elif pixels[x,y] != (0,0,0,0):
            pixels[x,y] = (0,0,0,0)
img.save('titles/tempest_clean.png')

